In [61]:
import numpy as np
from random import *
class NeuralNetwork:
    """Class that creates a NN and includes methods to train and test"""
    def __init__(self, setup=[[68,25,"sigmoid",0],[25,1,"sigmoid",0]],lr=.05,seed=1,error_rate=0,bias=1,iters=500,lamb=.00001,simple=0):
        #Note - these paramaters are examples, not the required init function parameters
        self._lr = lr
        self._seed = seed
        self._error_rate = error_rate
        self._bias = bias
        self._iters = iters
        self._lamb = lamb
        self._simple = simple
        

        # network is represented as a list of layers,
        # where layers are a list of nodes, where nodes
        # are a list of weights.
        # weights = [ [[w1,w2...], [w1,w2...]] <- layer1
        #             [[w1,w2...], [w1,w2...]] <- layer2
        #           ]
        weights = []
        outputs = []
        change = []
        
        # initialize the given number of layers with weights
        for layer in setup:
            weights.append(self.make_weights(n_inputs=layer[0],n_nodes=layer[1]))
            outputs.append([0] * layer[1])
            change.append([0] * layer[1])
        
        self._weights = weights
        self._outputs = outputs
        self._change = change
        
    @property
    def lr(self):
        return self._lr

    @lr.setter
    def lr(self, lr):
        self._lr = lr

    @property
    def bias(self):
        return self._bias

    @bias.setter
    def bias(self, bias):
        self._bias = bias 
    
    @property
    def seed(self):
        return self._seed

    @seed.setter
    def seed(self, seed):
        self._seed = seed 
    
    @property
    def outputs(self):
        return self._outputs

    @outputs.setter
    def outputs(self, outputs):
        self._outputs = outputs 
        
    @property
    def change(self):
        return self._change

    @change.setter
    def change(self, change):
        self._change = change 
    
    @property
    def weights(self):
        return self._weights

    @weights.setter
    def weights(self, weights):
        self._weights = weights 
        
        
    def make_weights(self,n_inputs, n_nodes):
        """
        Generates random weights for the network initialization

        Parameters
        ---------
        n_inputs
            Number of input nodes to this layer
        n_nodes
            Number of nodes to generate weights for
            
        Returns
        ---------
        Layer with random weights initialized for each node
        """
        seed(self.seed)
        layer = []
        
        # Get n_inputs random numbers between 0 and 1 for each node
        for i in range(n_nodes):
            node_weights = [random() for j in range(n_inputs)]
            node_weights.append(self.bias) # add bias at end
            layer.append(node_weights)
        
        return layer

    def feedforward(self, data):
        """
        Takes in data and passes it through the NN

        Parameters
        ---------
        data
            One datapoint
            
        Returns
        ---------
        The output(s) of the final layer in the network
        """
        inputs = data
        
        # pass data through all layers
        for layer in range(len(self.weights)):
            next_inputs = []
            for node in range(len(self.weights[layer])):
                sum = 0
                for i in range(len(inputs)): # multiply inputs by weights and add to sum
                    sum += inputs[i]*self.weights[layer][node][i]
                    
                sum += self.weights[layer][node][-1] # add bias
                output = sigmoid(sum) # Apply activation function
                self.outputs[layer][node] = output
                next_inputs.append(output)
            inputs = next_inputs
        
        # inputs should now be the final layer output
        return inputs
    
    def backprop(self, true_values, data):
        """
        Calculates the loss and gradient for each output node.
        Propagates the gradient through the network and records
        the error for each node.
        
        Parameters
        ---------
        true_values
            a list of true value(s) associated with the current
            training example

        Returns
        ---------
        None, change matrix is filled in for weight updating
        """
        # start at last layer
        for layer in reversed(range(len(self.outputs))): 
            if layer == len(self.outputs) - 1: # for last layer, calculate loss using true values
                for node in range(len(self.outputs[layer])):
                    loss = (true_values[node] - self.outputs[layer][node])
                    # fill in change matrix
                    self.change[layer][node] = loss*sigmoid_derivative(self.outputs[layer][node])
            else: # for all other layers
                for node in range(len(self.outputs[layer])):
                    loss = 0
                    # sum weighted losses from previous layer
                    for prev_layer_node in range(len(self.weights[layer + 1])):
                        loss += self.weights[layer+1][prev_layer_node][node]*self.change[layer+1][prev_layer_node]
                    # fill in change matrix
                    self.change[layer][node] = loss*sigmoid_derivative(self.outputs[layer][node])
         
        
        # Update weights
        for layer in reversed(range(len(self.outputs))): 
            input = data[-1] 
            if layer != 0: # the input to the first layer is the training example
                input = [self.outputs[layer][node] for node in range(len(self.outputs[layer - 1]))]
            for node in range(len(self.outputs[layer])):
                for i in range(len(input)):
                    self.weights[layer][node][i] += self.lr*self.change[layer][node]*input[i]
                # update bias
                self.weights[layer][node][-1] += self.lr*self.change[layer][node]
                

    def fit(self, training_data):
        """
        Calculates the loss and gradient for each output node.
        Propagates the gradient through the network and records
        the error for each node.
        
        Parameters
        ---------
        true_values
            a list of true value(s) associated with the current
            training example

        Returns
        ---------
        None, change matrix is filled in for weight updating
        """
        for epoch in range(n_epoch):
            sum_error = 0
            for row in training_data:
                output = feedforward(row)
                # Expected value should be last element of training row
                expected = row[-1]
                # Sum loss of all output nodes
                loss += sum([(expected[i]-output[i])**2 for i in range(len(expected))])
                backprop(network, expected, row)
            print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, loss))

    def predict(self, data):
        return feedforward(data)

def activation(input, weights):
    pass
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x)*(1 - sigmoid(x))

In [62]:

def test_make_weights():
    nn = NeuralNetwork(setup=[[8,3,"sigmoid",0],[3,8,"sigmoid",0]])
    assert len(nn.weights) == 2
    assert len(nn.weights[0]) == 3
    assert len(nn.weights[1]) == 8
    assert len(nn.weights[0][0]) == 9
    assert len(nn.weights[1][0]) == 4

def test_feedforward():
    nn = NeuralNetwork([[2,1, "sigmoid",0], [1,2, "sigmoid",0]])
    # a 2x1x2 network
    out = nn.feedforward([1,1])
    assert len(out) == 2
    assert nn.outputs[1] == out
    
def test_encoder():
    assert True

def test_encoder_relu():
    assert True

def test_one_d_ouput():
    assert True


In [63]:
nn = NeuralNetwork([[2,1, "sigmoid",0], [1,2, "sigmoid",0]])
# a 2x1x2 network
nn.feedforward([1,1])

[0.7536340857374849, 0.8512940040299428]

In [64]:
test_feedforward()
test_make_weights()


In [65]:
nn.weights

[[[0.13436424411240122, 0.8474337369372327, 1]],
 [[0.13436424411240122, 1], [0.8474337369372327, 1]]]

In [69]:
nn.backprop(true_values = [1,1], data = [[1,1], [1,1]])

In [67]:
nn.weights

[[[0.1347128032791722, 0.8477822961040037, 1.000348559166771]],
 [[0.13638443114440094, 1.0026805940312835],
  [0.8486085875694962, 1.001558913874117]]]

In [70]:
nn.feedforward([1,1])

[0.7552880993997311, 0.8519721417244803]

In [9]:
nn.weights


[[[0.13436424411240122, 0.8474337369372327, 1]],
 [[0.13436424411240122, 1], [0.8474337369372327, 1]]]

In [41]:
[nn.outputs[1][node] for node in nn.outputs[0]]

TypeError: list indices must be integers or slices, not numpy.float64

In [42]:
nn.outputs[0]

[0.8788726974657645]

In [21]:
nn.change[1][1]

0.031178277482338014

In [22]:
nn.weights[1][1]

[0.8474337369372327, 1]

In [60]:
test = [[1,1], [1,1]]
test[-1]

[1, 1]